In [ ]:

# 플레이어(에이전트) 정의
#  - 상태(state): (현재 합, 사용 가능한 에이스 여부, 딜러 오픈카드)
#  - 행동(action): Hit(True) / Stand(False)
#  - 보상(reward): 승(+1), 패(-1), 무승부(0) - 에피소드가 끝난 뒤 한 번에 부여import numpy as np
import matplotlib.pyplot as plt

class Player(object):
    def __init__(self, currentSum, usableAce, dealersCard):
    #currentSum : 플레이어가 들고 있는 카드의 합
    #usableAce : 플레이어가 에이스를 11로 쓸 수 있는 상태인지 여부
    #dealersCard : 딜러가 공개한 카드 값
    #self.usingAce : 에이스 1로 쓰는지 11로 쓰는지.

        self.currentSum = currentSum
        self.dealersCard = dealersCard
        self.usableAce = usableAce
        self.usingAce = self.usableAce

    def ReceiveCard(self, card):
        if self.usingAce and self.currentSum + card > 21:
            self.usingAce = False
            self.currentSum += card - 10
        else:
            self.currentSum += card

 #새 카드를 받았을 때 합 업데이트
 #에이스 사용중인데 새 카드더해서 21 넘으면 에이스 1로 쓰기

    def GetState(self):
        return (self.currentSum, self.usableAce, self.dealersCard)
        # 상태를 (현재합, usableAce(보유 여부), 딜러 오픈카드) 튜플로 반환

    def GetValue(self):
        return self.currentSum

    def ShouldHit(self, policy):
        return policy[self.GetState()]

    def Bust(self):
        return self.GetValue() > 21
        # 21 초과하면 죽음

class Dealer(object):
    def __init__(self, cards):
        self.cards = cards

    def ReceiveCard(self, card):
        self.cards.append(card)

    def GetValue(self):
        currentSum = 0
        aceCount = 0

        for card in self.cards:
            if card == 1:
                aceCount += 1
            else:
                currentSum += card
# 에이스 처리(가능하면 11로, 넘치면 1로)
        while aceCount > 0:
            aceCount -= 1
            currentSum += 11

            if currentSum > 21:
                aceCount += 1
                currentSum -= 11
                currentSum += aceCount
                break

        return currentSum

    def ShouldHit(self):
        if self.GetValue() >= 17:
                # 17 이상이면 스탠드, 그 외에는 히트
            return False
        else:
            return True

    def Bust(self):
        return self.GetValue() > 21

class StateActionInfo(object):
    def __init__(self):
        self.stateActionPairs = [ ]
        self.stateActionMap = set()

    def AddPair(self, pair):
        if pair in self.stateActionMap:
            return  # 이미 기록된 (s,a)이면 스킵

        self.stateActionPairs.append(pair)
        self.stateActionMap.add(pair)

def EvaluateAndImprovePolicy(qMap, policy, returns, stateActionPairs, reward):
    for pair in stateActionPairs:
        returns[pair] += 1
        qMap[pair] = qMap[pair] + ((reward - qMap[pair]) / returns[pair])

        state = pair[0]
        shouldHit = False
#평균 업데이트: Q ← Q + (R - Q) / N
        if qMap[(state, True)] > qMap[(state, False)]:
            shouldHit = True

        policy[state] = shouldHit

def newCard():
    card = np.random.randint(1, 14)

    if card > 9:
        return 10
    else:
        return card

def PlayEpisode(qMap, policy, returns):
# 무작위 초기 상태
    playerSum = np.random.randint(11, 22)
    dealerOpenCard = np.random.randint(1, 11)
    usableAce = bool(np.random.randint(0, 2))

    player = Player(playerSum, usableAce, dealerOpenCard)
    dealer = Dealer([dealerOpenCard])

    stateActionInfo = StateActionInfo()

        # 초기 행동도 무작위 선택(탐색 보장)
    hitAction = bool(np.random.randint(0, 2))
    stateActionInfo.AddPair((player.GetState(), hitAction))

    # 플레이어 턴: 초기 행동이 Hit이면 카드를 받고,
    # 정책(policy)에 따르면 계속 Hit할 수 있음

    if hitAction:
        player.ReceiveCard(newCard())

        while not player.Bust() and player.ShouldHit(policy):
            stateActionInfo.AddPair((player.GetState(), True))
            player.ReceiveCard(newCard())

  # 플레이어가 버스트면 즉시 패배
    if player.Bust():
        EvaluateAndImprovePolicy(qMap, policy, returns, stateActionInfo.stateActionPairs, -1)
        return

    stateActionInfo.AddPair((player.GetState(), False))
    dealer.ReceiveCard(newCard())

    while not dealer.Bust() and dealer.ShouldHit():
        dealer.cards.append(newCard())

   # 최종 보상 산정(+1/0/-1) 후, 해당 에피소드에서 방문한 모든 (s,a)에 동일 보상 반환
    if dealer.Bust() or dealer.GetValue() < player.GetValue():
        EvaluateAndImprovePolicy(qMap, policy, returns, stateActionInfo.stateActionPairs, 1)
    elif dealer.GetValue() > player.GetValue():
        EvaluateAndImprovePolicy(qMap, policy, returns, stateActionInfo.stateActionPairs, -1)
    else:
        EvaluateAndImprovePolicy(qMap, policy, returns, stateActionInfo.stateActionPairs, 0)


qMap = { }
policy = { }
returns = { }

for playerSum in range(11, 22):
    for usableAce in range(2):
        for dealersCard in range(1, 11):
            playerState = (playerSum, bool(usableAce), dealersCard)
            qMap[(playerState, False)] = 0
            qMap[(playerState, True)] = 0
            returns[(playerState, False)] = 0
            returns[(playerState, True)] = 0

            if playerSum == 20 or playerSum == 21:
                policy[playerState] = False
            else:
                policy[playerState] = True
# Monte-Carlo 컨트롤 루프: 충분히 많은 에피소드를 돌며
#  정책을 점차 개선(탐욕적 정책 개선) -> 수렴 기대
for i in range(100000):
    PlayEpisode(qMap, policy, returns)
# 시각화: (딜러 오픈, 플레이어 합) 평면에서
#  - 파란 점: Hit
#  - 노란 점: Stand
#  - 에이스 사용 가능 여부에 따라 두 그림으로 분리
x11 = [ ]
y11 = [ ]

x12 = [ ]
y12 = [ ]

x21 = [ ]
y21 = [ ]

x22 = [ ]
y22 = [ ]

for playerState in policy:
    if playerState[1]:
        if policy[playerState]:
            x11.append(playerState[2] - 1)
            y11.append(playerState[0] - 11)
        else:
            x12.append(playerState[2] - 1)
            y12.append(playerState[0] - 11)
    else:
        if policy[playerState]:
            x21.append(playerState[2] - 1)
            y21.append(playerState[0] - 11)
        else:
            x22.append(playerState[2] - 1)
            y22.append(playerState[0] - 11)

plt.figure(0)
plt.title('With Usable Ace')
plt.scatter(x11, y11, color='blue')
plt.scatter(x12, y12, color='yellow')
plt.xticks(range(10), [ 'A', '2', '3', '4', '5', '6', '7', '8', '9', '10' ])
plt.yticks(range(11), [ '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21' ])

plt.figure(1)
plt.title('Without Usable Ace')
plt.scatter(x21, y21, color='blue')
plt.scatter(x22, y22, color='yellow')
plt.xticks(range(10), [ 'A', '2', '3', '4', '5', '6', '7', '8', '9', '10' ])
plt.yticks(range(11), [ '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21' ])

plt.show()